Cylleneus
=========

## Working with a new corpus
Cylleneus comes with pre-made infrastructure for working with several different Sanskrit, Greek, and Latin corpora.
To get the search engine working with a new corpus, new infrastructure will need to be built for indexing source files, for displaying text, and (if needed) for handling new query types. Very specifically, the following will need to be created, depending on the kinds of information provided by the corpus:

1. A ``DocumentSchema`` defining the available search fields for corpus documents;
2. A ``Tokenizer`` which parses the document text to be fed into the indexing pipeline;
3. A ``LemmaFilter``, ``SynsetFilter``, ``AnnotationFilter``, ``SemfieldFilter`` and optionally a ``MorphosyntaxFilter`` for gathering data and preparing it in the correct formats for insertion into the index. For plaintext corpora, the pre-built defaults in ``corpus.default`` are likely sufficient, and for many structured corpora probably only a bespoke ``LemmaFilter`` will be required;
4. A ``fetch`` function for retrieving text from source files for display.


In [ ]:
# Utility imports
from pprint import pprint
from random import choice

# For purposes of an example
from cylleneus.corpus import Corpus
agldt = Corpus("agldt")
if not agldt.searchable:
    agldt.download()

## Initial set-up

Every available corpus type (independent of whether indexes for the corpus are present or not) is represented by a package within the directory hierarchy of ``cylleneus\corpus``. To allow Cylleneus to recognize the package as a corpus module, certain files with specific contents must be present. ``__init__.py`` declares metadata for inclusion in the corpus manifest.

###### cylleneus\\corpus\\<language\>\\<name\>\\__init__.py
```
from .core import description, language, glob, repo, fetch
from .preprocessor import Preprocessor
from .schema import DocumentSchema
from .tokenizer import Tokenizer

from cylleneus.corpus.meta import CorpusMeta

# Manifest information
meta = CorpusMeta(
    description,
    language,
    DocumentSchema,
    Tokenizer,
    Preprocessor,
    glob,
    fetch,
    repo,
)
```

All attributes of the ``CorpusMeta`` object must be given values, even if some of these attributes (e.g., ``Tokenizer``) utilize the provided defaults.

In [ ]:
# E.g.,

from cylleneus.corpus.lat.agldt import meta as corpusmeta
pprint(corpusmeta)

## The document schema
The ``DocumentSchema`` class implements the interface between corpus source files and Cylleneus' indexing pipeline and query system. A schema defines the metadata that will be stored for each indexed document and defines the fields -- and consequently the query types -- that are supported by a corpus.

### Standard fields
Every schema inherits the fields ``corpus``, ``docix``, ``author``, ``title``, ``language``, ``filename`` and ``datetime`` from the schema base class, which are plaintext (``STORED``) fields for storing basic document metadata. The values for these fields should be set during preprocessing, before any text data is passed to the indexing pipeline(``docix`` is computed automatically by the document indexer; it does not need to be set during preprocessing).

The schema should then define:

1. A ``urn`` (``STORED``) field for storing a unique identification tag for the document (ideally, the CTS URN of the work represented by the document);
2. A ``meta`` (``STORED``) field defining the section divisions of the text;
3. The fields ``form`` (of the ``FORM`` field class), ``lemma`` (``LEMMA``), ``synset`` (``SYNSET``), and ``semfield`` (``SEMFIELD``) corresponding to Cylleneus' query types;
4. Any optional fields such as ``morphosyntax`` (of the ``MORPHOSYNTAX`` field class), depending on the corpus' annotation structure.

###### cylleneus\\corpus\\<language\>\\<name\>\\schema.py
```
from cylleneus.engine.fields import *
from cylleneus.engine.schemas import BaseSchema
from cylleneus.corpus.default import CachedTokenizer
from cylleneus.engine.analysis.filters import (
    CachedLemmaFilter,
    CachedSynsetFilter,
    AnnotationFilter,
    SemfieldFilter,
)

Tokens = CachedTokenizer(chars=True)
Lemmas = CachedLemmaFilter(chars=True)
Synsets = CachedSynsetFilter()
Annotations = AnnotationFilter()
Semfields = SemfieldFilter()


class DocumentSchema(BaseSchema):
    . . .
    form = FORM(analyzer=Tokens, vector=True)
    lemma = LEMMA(analyzer=Tokens | Lemmas, vector=True)
    annotation = ANNOTATION(
        analyzer=Tokens | Lemmas | Annotations, vector=True
    )
    synset = SYNSET(analyzer=Tokens | Lemmas | Synsets, vector=True)
    semfield = SEMFIELD(
        analyzer=Tokens | Lemmas | Synsets | Semfields, vector=True
    )
```

The Filters constitute a pipeline for each field type, through which a stream of ``Tokens`` objects is fed, undergoing transformations at each step for storage in the index. Thus, the raw form generated by the ``Tokenizer`` is transformed by the ``LemmaFilter`` into a lemma; the lemma is then used by the ``SynsetFilter`` to retrieve the word's synsets from the WordNet; the synsets are used by the ``SemfieldFilter`` to retrieve the corresponding semfields; and so on. (For all components of the pipeline that are shared between fields -- namely, the ``Tokenizer``, ``LemmaFilter``, and ``SynsetFilter`` -- data is cached for efficiency).

### Other fields
Additional ``STORED`` fields can be defined to store any other relevant document-specific information. For example, a translation alignment corpus might wish to store the name of the translator of the work along with its author. New field types along with new query specifications can also be built for specialized annotations. Because syntactically annotated corpora are fairly common -- and an important use case for Cylleneus -- the library comes with a pre-built ``MORPHOSYNTAX`` field for corpora which contain syntactic annotations (permitting queries via ``/ . . . /``). To use this field, ``schema.py`` would need to include:

###### cylleneus\\corpus\\<language\>\\<name\>\\schema.py
```
class DocumentSchema(BaseSchema):
    ...
    morphosyntax = MORPHOSYNTAX(analyzer=Tokens | Morphosyntax, vector=True)
```

However, because syntactic annotations are not standardized across corpora and may reflect different classification systems (or indeed, entirely different theories of syntax), a bespoke ``MorphosyntaxFilter`` may need to be created, although in most cases the value of annotations can be directly indexed without further transformation. (The default ``MorphosyntaxFilter`` found in ``cylleneus.engine.analysis.filters`` can be easily modified to suit the particular annotation structure of a new corpus).

In [ ]:
# E.g.,

from cylleneus.corpus.lat.agldt import DocumentSchema
schema = DocumentSchema()
pprint(schema.items())

## Basic corpus metadata

Corpus metadata is defined in ``core.py``. Minimally, this file must include:

###### cylleneus\\corpus\\lat\\my_corpus\\core.py
```
# A descriptive title
description = ""

# The (main) language of the corpus ("skt", "grk" or "lat")
language = ""

# Glob pattern (for identifying source files)
glob = ""

# Repo (fully qualified URLs for the git repository and raw file storage of a remotely-accessible corpus)
repo = {
    "origin":   "",
    "raw":      "",
    "location": ""  # "remote" or "local"
}


# Function to fetch text from corpus source files
def fetch(work, meta, fragment):
    ...
```


Search results can be passed to a corpus' ``fetch`` function in order to obtain highlighted display text. This is required because Cylleneus does *not* store text in the index itself, in order to keep index size to a minimum as
well as to permit Cylleneus to function as a simple reference retrieval system, leaving text display functionality as a separate concern. For more information about the ``fetch`` function, see below.

## Document preprocessing

The ``Preprocessor`` class characterizes the direct interface between a corpus source file and the indexing pipeline. Essentially, it tells Cylleneus how to parse a file in order to retrieve the information required for the fields of the ``DocumentSchema``. Its single method ``parse`` takes a file path and returns a ``dict`` whose keys correspond to the names of the fields specified in the schema. The value given for the ``"form"``, ``"lemma"``, ``"synset"``, and ``"semfield"`` keys should be the raw text data, in the appropriate format for the corpus (XML, JSON, plaintext, etc.), to be handled by the tokenizer and filters. 

###### cylleneus\\corpus\\<language\>\\<name\>\\preprocessor.py

```
class Preprocessor(BasePreprocessor):
    def parse(self, file: Path):
        # Read in the source file; parse values for "author", "title", "urn", "meta", etc.
        ...
```

NB. The value of ``"meta"`` must be string defining the standardized labels for the section divisions of the text, beginning at the highest level of organization. The labels should be given all in lowercase and separated by a dash (``-``); e.g. ``"book-line"`` or ``"book-chapter-section"``.

In [ ]:
# E.g.,

from cylleneus.corpus.lat.agldt import Preprocessor
preprocessor = Preprocessor("agldt")

file = choice(list(agldt.text_dir.glob(agldt.glob)))
parsed = preprocessor.parse(file)
pprint(parsed)

## The indexing pipeline

### Bird's-eye view
The indexing pipeline consists of a ``Tokenizer`` that breaks the source text into discrete tokens (however defined by the corpus), which store the text and other information for processing and storing in the index by a series of Filters. The ``Tokenizer`` is responsible for parsing referencing information to be stored in the ``meta`` attribute of the ``Token`` object. In almost all cases, the pipeline will then include:

1. A ``LemmaFilter``, which processes information for lemma-based searching via the index's ``lemma`` field, and which feeds data onward to the rest of the pipeline. For unstructured (plaintext) corpora, the ``LemmaFilter`` should transform raw forms into lemmas using the WordNet's lemmatization service. For structured corpora, the ``LemmaFilter`` can use provided lemmatization, on relying on the WordNet for more detailed lexical and morphological information;
2. A ``SynsetFilter``, which uses the lemmas to fetch word senses (synsets) from the WordNet, along with other semantic information;
3. A ``SemfieldFilter``, for processing semantic domain information for synsets;
4. An ``AnnotationFilter``, which processes morphological annotations;
5. An (optional) ``MorphosyntaxFilter``, for indexing syntactic annotations, if provided;
6. Any other bespoke Filters for indexing corpus-specific mark-up.

NB. For memory efficiency, each component of the pipeline operates over the same series of ``Token`` objects, modifying the properties of a token at each step. The relevant data for indexing by each Filter is set by the ``text`` property, but Filters may define other attributes on the ``Token`` in order to pass data on to the next Filter.

### Tokenization
The first step of the indexing process is tokenization. The ``Tokenizer`` class performs this task, generating a sequence of ``Token`` objects representing the document text. It is the responsibility of the ``Tokenizer`` to track precise reference citation information for all tokens, stored in a ``meta`` attribute. In addition, all tokenizers should track global position character position -- ``pos``, ``startchar`` and ``endchar`` -- stored directly on the ``Token`` object, for each token. The ``meta`` attribute is a ``dict``; one key should always be ``meta``, describing the labels of the work's section divisions (e.g., ``"book-poem-line"``). There should be values for each of the work's section divisions (e.g., ``{ . . . "book": "1", "poem": "1", "line": "1" }``), as well as for: "``sect_sent``", the sentence number within the current division (beginning from 0); "``sect_pos``", the global token position within the current division (beginning from 0); "``sent_id``", the global sentence number within the work as a whole (beginning from 0); "``sent_pos``", the token number within the current sentence (beginning from 0).

###### cylleneus\\corpus\\<language\>\\<name\>\\tokenizer.py

```
import copy

from cylleneus.engine.analysis.tokenizers import Tokenizer
from cylleneus.engine.analysis.acore import CylleneusToken


class CachedTokenizer(Tokenizer):
    def __init__(self, cached=True, **kwargs):
        super(CachedTokenizer, self).__init__()
        self.cached = cached
        self._cache = None
        self._docix = None
        self.__dict__.update(**kwargs)

    @property
    def cache(self):
        return copy.deepcopy(self._cache)

    def __call__(
        self,
        data,
        positions=True,
        chars=True,
        keeporiginal=True,
        removestops=True,
        tokenize=True,
        start_pos=0,
        start_char=0,
        mode="",
        **kwargs,
    ):
        if kwargs.get("docix", None) == self._docix and self._cache:
            yield from self.cache
        else:
            t = CylleneusToken(
                positions, chars, removestops=removestops, mode=mode, **kwargs
            )

            if t.mode == "query":
                t.original = t.text = data
                yield t
            else:
                self._cache = []
                self._docix = kwargs.get("docix", None)

                if not tokenize:
                    t.text = t.original = data
                    t.boost = 1.0
                    if positions:
                        t.pos = start_pos
                    if chars:
                        t.startchar = start_char
                        t.endchar = start_char + len(t.original)
                    yield t
                else:
                    for sentence in data:
                        # Track "sent_id" and "sect_sent"
                        ...
                        for word in sentence:
                            # Track "sect_pos" and "sent_pos", "pos", "startchar", "endchar"
                            ...
                            if self.cached:
                                self._cache.append(copy.deepcopy(t))
                            yield t
```

``meta`` is not the only attribute that should be set on the ``Token`` object. The relevant string to be indexed at any point in the indexing process (at the lemma, syntax or semfield stage) must be stored in the ``text`` attribute. If any text manipulation is performed (e.g., Unicode normalization or editorial modification), the raw form should be stored as ``original``. If the corpus contains annotations, for instance lemmatization or morphological tagging, this data should be stored on the ``Token`` object for processing by the appropriate Filter: ``lemma``, for unprocessed lemmatization data; ``synset``, for unprocessed semantic information, if available; ``morpho``, for morphological annotations, preferably conforming to Cylleneus' morphological tagging scheme; ``morphosyntax``, for unprocessed syntactic annotations.

Other attributes can be set on the ``Token`` object for processing by custom-made Filters.

In [ ]:
# E.g.,

from cylleneus.corpus.lat.agldt.tokenizer import CachedTokenizer
tokens = CachedTokenizer()
doc = parsed["form"]

# for token in tokens(doc, mode="index", docix=0):
#    pprint(token)

### Lemma filtering

The ``LemmaFilter`` transforms raw word forms into lemmas, either directly through lemmatization via the WordNet, or relying on annotations provided by the corpus to fetch the relevant lemma record from the WordNet. Using the retrieved lexical data, the ``Token``'s ``text`` attribute must be set to a string value of the format ``"lemma:uri=morpho"``. (In cases where lemmatization is ambiguous, discrete tokens should in fact be generated for for all possible lemmas. For example, the form *dices* would be lemmatized to either *dico, dicare* or *dico, dicere*, thus separate ``Token``s for each lemma will need to be generated with the appropriate morphological tag).

Furthermore, because the lemma's base morphological descriptor is now available, the ``LemmaFilter`` should also transform the value of the token's ``morpho`` attribute, into the format ``"morpho::uri:n>annotation"``, where ``morpho`` is the lemma's base morphological descriptor; ``uri`` is the lemma's URI; ``n`` is an index for cases of ambiguous lemmas (otherwise ``0``); and ``annotation`` is the morphological descriptor provided derived from mark-up or via WordNet lemmatization. If annotation or lemmatization provides multiple annotations (e.g., the form ``virtutibus`` would yield ``"n-p---fd3-"`` as well as ``"n-p---fb3-"``, these annotations should be conjoined as a single space-separated string.

In [ ]:
# E.g., 

from cylleneus.corpus.lat.agldt.filters import CachedLemmaFilter
lemmas = CachedLemmaFilter()
doc = parsed["lemma"]

# for token in lemmas(tokens(doc, mode="index", docix=0)):
#     pprint(token)

### Annotation filtering

The ``AnnotationFilter`` takes the ``Token``'s value of ``morpho`` and emits a series of new ``Token`` objects for every possible morphological value in each tag. Provided that Filters occurring earlier in the pipeline have provided data of the correct form, the default ``AnnotationFilter`` should be sufficient for almost every case.

In [ ]:
# E.g., 

from cylleneus.engine.analysis.filters import AnnotationFilter
annotations = AnnotationFilter()
doc = parsed["annotation"]

# for token in annotations(lemmas(tokens(doc, mode="index", docix=0))):
#    pprint(token)

### Synset filtering

Based on the information provided by the ``LemmaFilter``, the ``SynsetFilter`` gathers semantic information from the WordNet and will emit discrete ``Token`` objects for each the word's synsets. For every synset, the ``Token``'s ``text`` attribute should be set to the corresponding value in the form ``<pos>#<offset>``. (To reduce API traffic, the ``Token`` object's ``code`` attribute should also be set to a space-separated string capturing the synset's semfield information in the form of Dewey Decimal classification numbers.

In [ ]:
# E.g.,

from cylleneus.engine.analysis.filters import CachedSynsetFilter
synsets = CachedSynsetFilter()
doc = parsed["synset"]

# for token in synsets(lemmas(tokens(doc, mode="index", docix=0))):
#    pprint(token)

### Semfield filtering

Using the value of a ``Token`` object's ``code`` attribute set by the ``SynsetFilter``, the ``SemfieldFilter`` emits a new ``Token`` for each semfield code number, setting its ``text`` attribute to the relevant value.

In [ ]:
# E.g., 

from cylleneus.engine.analysis.filters import SemfieldFilter
semfields = SemfieldFilter()
doc = parsed["semfield"]

# for token in semfields(synsets(lemmas(tokens(doc, mode="index", docix=0)))):
#    pprint(token)

### Morphosyntax filtering

For structured corpora with syntactic annotations (e.g., treebanks), the ``morphosyntax`` property of the ``Token`` object should be set during tokenization. It is then the task of the ``MorphosyntaxFilter`` to extract this value and emit new ``Token``s whose ``text`` value has been set to the relevant value.

In [ ]:
# E.g., 

from cylleneus.corpus.lat.agldt.filters import MorphosyntaxFilter
morphosyntax = MorphosyntaxFilter()
doc = parsed["morphosyntax"]

# for token in morphosyntax(tokens(doc, mode="index", docix=0)):
#    pprint(token)

## Fetching and highlighting text

Though not strictly part of the indexing pipeline, a corpus ``fetch`` function is required for retrieving and highlighting the text that corresponds to any result matching for a query in the index. The ``fetch`` function, defined in ``core.py``, takes three arguments, which will be passed to it by a ``Search`` object (representing the results of a query):

``work``, a ``Work`` object representing the specific document in which a query has been matched;
``meta``, containing reference citation metadata for the match;
and
``fragment``, which in almost every instance will be a null value; this serves as a placeholder for non-standard situations where an index *does* store textual data.

Using the reference citation metadata in ``meta``, the ``fetch`` function retrieves the relevant text -- and context -- from the document represented by ``work``. (Keep in mind that in general a work as traditionally conceived may be represented by multiple indexes -- multiple documents. The ``Work`` object captures this abstraction; however, in the context of search results, the ``Work`` object will point only to the single document in which a query has been matched).

It is the job of the ``fetch`` function to highlight as well as to retrieve the matching text, along with a certain amount of surrounding context (specified in ``settings.py`` as ``LINES_OF_CONTEXT``: how this is interpreted for a particular corpus will depend on the structure of the text). Very specifically, the ``fetch`` function needs to return a tuple consisting of:

``urn``, a uniquely identifying descriptor of the work, ideally a CTS URN;
``reference``, a formatted string giving the reference citation in human-readable form (e.g.
"book: 3, line: 567"; for plaintext corpora without annotated section divisions, the ``startchar`` and ``endchar`` of the matching text are available;
``text``, the matching text and context, including where possible highlighted search terms.

The returned ``text`` value has a specific make-up and format which can be used for highlighting and display. The ``meta`` parameter (a ``dict``) will contain highlighting information, accessible by the key ``"hlites"``. This should be used to provide pinpoint highlighting of matched terms.

Pre-match and post-match surrounding context (however defined for the corpus) should be wrapped in the tags ``<pre> . . . </pre>`` and ``<post> . . . </post>`` (each unit, e.g., line, of context should be tagged in this way). The matching text itself should be wrapped in the tag ``<match> . . . </match>`` and the specific matching terms should be highlighted with the tag ``<em> . . . </em>``. This provides a standarized mark-up scheme for applications which want to present the text to users in a prettily formatted way (e.g., a reading environment).


In [ ]:
# A purely fabricated example; normally, the values of work, meta and fragment
# would be generated programmatically during query search
work = list(agldt.works)[0]  # Cicero, *In Catilinam*
meta = {'meta': 'chapter-section', 'hlites': [{'chapter': '2', 'section': '5', 'sent_id': '250', 'sent_pos': '27'}], 'start': {'chapter': '2', 'section': '5', 'sect_sent': None, 'sect_pos': None, 'sent_id': '250', 'sent_pos': '27', 'startchar': 23492, 'endchar': 23500, 'pos': 26}, 'end': {'chapter': '2', 'section': '5', 'sect_sent': None, 'sect_pos': None, 'sent_id': '250', 'sent_pos': '27', 'startchar': 23492, 'endchar': 23500, 'pos': 26}}
fragment = None
fetch = corpusmeta.fetch
pprint(fetch(work, meta, fragment))

## Corpus management

Once the necessary infrastructure for working with a corpus is in place, indexes can be built. Corpus management tasks, including indexing, are most easily managed using the ``cylleneus`` CLI script. To create indexes for a corpus from a set of source files, these files first need to be placed in the correct directory and folder hierarchy. To discover where, on a particular machine and operating system, this directory should reside, you can use:

``$ cylleneus where``

In the relevant directory, you will find separate folders for supported languages: ``skt``, ``grk``, ``lat``, ``eng`` (used for translation alignments). In the appropriate language directory, create a new folder using the label of the corpus: e.g., ``perseus``, ``lasla``, ``diorisis``, ``dcs``. This must be the same as the name of the folder in which the corpus module resides.

Within the corpus directory, create a folder ``text`` to store source files; ``index`` (automatically created during indexing if it does not already exist) will hold corresponding index files. Place the source files you wish to index directly in the ``text`` folder.

From the command line, you can now use:

``$ cylleneus create --corpus <name>``

to begin indexing. The option ``--destructive`` will clear the corpus directory and manifest prior to indexing; this starts fresh; ``--optimize`` will further optimize the index files after create (optimization can be performed independently after indexing, as well).

The indexing process automatically creates a file ``manifest.json`` in the corpus directory to store corpus-specific index data. This data is available on the ``Corpus`` object as ``manifest``.

In [ ]:
# E.g.,
pprint([agldt.path, agldt.text_dir, agldt.index_dir])
pprint(agldt.manifest)